In [1]:
from transformers import BertTokenizer, BertForTokenClassification, pipeline, get_linear_schedule_with_warmup
from transformers import CanineTokenizer, CanineForTokenClassification
from seqeval.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
import torch
import os
import re


In [2]:
from custom_train_loop_char_level import load_model, load_data, valid

In [3]:
label2id = {'B-width': 1,
        'B-height': 2,
        'B-radius': 3,
        'B-brand': 4,
        'B-line': 5,
        'O': 0}

id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'B-brand', 
            5: 'B-line', 
            0: 'O'}

# model_pth = "/home/sondors/29887"
# model_pth = '/home/sondors/weights/epoch_5'
model_pth = "/home/sondors/Documents/price/BERT_NER/weights/epoch_1"
device = 'cuda'

# AutoModelForSequenceClassification.from_pretrained("path/to/model")
# tokenizer, model = load_model(model_pth, device, label2id, id2label)

model = CanineForTokenClassification.from_pretrained(model_pth).to(device)
tokenizer = CanineTokenizer.from_pretrained(model_pth)


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [4]:
pth = '/home/sondors/Documents/price/BERT_NER/csv/NER_2609.csv'
max_seq_length = 512
train_batch_size = 2
val_batch_size = 2
train_size = 0.8

training_loader, testing_loader = load_data(tokenizer, pth, train_size, label2id, train_batch_size, val_batch_size, max_seq_length)

FULL Dataset: (7018, 8)
TRAIN Dataset: (5614, 8)
TEST Dataset: (1404, 8)


In [5]:
report = valid(model, testing_loader, id2label, device)
print(f"{report}\n")

Validation loss per 100 evaluation steps: 0.024174122139811516
Validation loss per 100 evaluation steps: 0.012416514369778999
Validation loss per 100 evaluation steps: 0.012207454749248663
Validation loss per 100 evaluation steps: 0.011965241505953165
Validation loss per 100 evaluation steps: 0.011988035636022687
Validation loss per 100 evaluation steps: 0.012040466359703097
Validation loss per 100 evaluation steps: 0.011899889507230229
Validation loss per 100 evaluation steps: 0.011766291236752026
Validation Loss: 0.011754983414542938
Validation Accuracy: 0.9686159492935799
              precision    recall  f1-score   support

       brand       0.97      0.99      0.98      9172
      height       0.99      0.97      0.98      2472
        line       0.96      0.14      0.25      1227
      radius       0.95      0.98      0.96      2898
       width       0.91      0.99      0.95      4202

   micro avg       0.95      0.93      0.94     19971
   macro avg       0.96      0.81     

In [6]:
def price_data_raw():
    pth = "/home/sondors/Documents/price/BERT_NER/csv/list1.xlsx"
    df = pd.read_excel(pth)

    clms = ['PRICE_FRNO', 
        'PRICE_FNAM', 
        'PRICE_CAID', 
        'PRICE_CNAM',  
        'PRICE_TMNO', 
        'PRICE_PRCN', 
        'PRICE_VERT', 
        'PRICE_PCOD', 
        'date', 
        'PRICE_IMGN', 
        'PRICE_LOCD', 
        'lineNumber',
        'PRICE_PRDS',
        'PRICE_DPRN',
        'PRICE_PTYP',
        'PRICE_PROMO',
        'PRICE_CURR',
        'PRICE_LOCA',
        'PRICE_WARR',
        'PRICE_RSLT',
        'PRICE_PLID',
        'PRICE_SALES',
        'PRICE_DTSH',
        'PRICE_BARCODE',
        'комментарий',
        'kpl',
        'PRICE_PARAM',
        'PRICE_NOTE']

    clms_hz = ['kpl',
        'PRICE_NOTE',
        'PRICE_PARAM']

    df = df.drop(columns=clms + clms_hz)
    return df
df = price_data_raw()
df

,PRICE_NAME,Ширина,Высота,Радиус,Инд. Скорост,XL,RFT,Линейка,ppl
0,"10,5/31 R15 Bridgestone Dueler A/T 001 109S",NaN,NaN,NaN,109S,NaN,NaN,Dueler,Bridgestone
1,12067 Bridgestone Автошина R20 275/50 Bridgest...,275,50,20,113R,XL,NaN,Blizzak DM-V2,Bridgestone
2,12088 BRIDGESTONE Автошина R20 255/50 Bridgest...,255,50,20,109T,XL,NaN,Blizzak DM-V2,Bridgestone
3,13/175/70 Bridgestone Ice Cruiser 7000S 82T ш,175,70,17,82S,NaN,NaN,Ice Cruiser 7000S,Bridgestone
4,13602 BRIDGESTONE Автошина R18 275/35 Bridgest...,275,35,18,95S,NaN,NaN,Blizzak Ice,Bridgestone
...,...,...,...,...,...,...,...,...,...
23143,"Бриджстоун 2657015 T 112 AT001, BRIDGESTONE 12...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bridgestone
23144,"Бриджстоун 2753518 S 95 VRX, BRIDGESTONE 8393 ...",NaN,NaN,NaN,NaN,NaN,NaN,Blizzak VRX,Bridgestone
23145,Бриджстоун 315/70r22.5 R249 Tl 152/154 L/M Рул...,315,70,15,152/154,NaN,NaN,NaN,BRIDGESTONE
23146,шинаBridgestone LT245/75R16 120Q Dueler M/T 674,245,75,16,120Q,NaN,NaN,Dueler,Bridgestone


In [7]:
offer_1 = df.iloc[23145]['PRICE_NAME']
offer_1

'Бриджстоун 315/70r22.5 R249 Tl 152/154 L/M Рулевая Bridgestone 12646'

In [8]:
df.iloc[1]

PRICE_NAME      12067 Bridgestone Автошина R20 275/50 Bridgest...
Ширина                                                        275
Высота                                                         50
Радиус                                                         20
Инд. Скорост                                                 113R
XL                                                             XL
RFT                                                           NaN
Линейка                                             Blizzak DM-V2
ppl                                                   Bridgestone
Name: 1, dtype: object

In [9]:
def inference(offer, model, tokenizer):
    pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
    return pipe(offer)

def brand_line_width_height_radius(result):
    # Создаем словари для каждой сущности
    entities = {}
    for item in result:
        entity_group = item['entity_group']
        word = item['word']
        score = item['score']
        if score > 0.7:
            if entity_group in entities:
                entities[entity_group].append(word)
            else:
                entities[entity_group] = [word]

    # Формируем строки для каждой сущности
    brand_str = ''.join(entities.get('brand', []))
    width_str = ''.join(entities.get('width', []))
    height_str = ''.join(entities.get('height', []))
    radius_str = ''.join(entities.get('radius', []))
    line_str = ''.join(entities.get('line', []))

    # Выводим строки
    print(f"Brand: {brand_str}")
    print(f"Width: {width_str}")
    print(f"Height: {height_str}")
    print(f"Radius: {radius_str}")
    print(f"Line: {line_str}")
    return brand_str, line_str, width_str, height_str, radius_str

def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    pattern1 = r'([rR])(\d)'
    processed_text = re.sub(pattern1, r'R \2', input_text)

    pattern2 = r'(\d)([rR])'
    processed_text = re.sub(pattern2, r'\1 R', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")

    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

result = inference(offer_1, model, tokenizer)
brand_line_width_height_radius(result)
brand_line_width_height_radius(inference(process_text(offer_1), model, tokenizer))


Brand: Бриджстоун
Width: 31522
Height: 
Radius: 
Line: 
Brand: Бриджстоун
Width: 152
Height: 
Radius: 
Line: 


('Бриджстоун', '', '152', '', '')